### Problem statement
YouTube is the most popular video website in the world. They mantain a list of the top trending videos of every day which are selected by their own algorithms based on many factors of a video aside from just the number of views. As a YouTube user, it would be very interesting to find out insights about the trending and hopefully come out patterns that can help YouTubers to be more successful with their channels. Having considered that there is no suitable target variables in the dataset, thus, in this project, I would only focus on data visualization.

### Data
* The dataset is a csv file conataing a list of the trending YouTube videos from 2017-11 to 2018-06 in the US and Canada including the video title, channel title, category_id, publish time, tags, views, likes and dislikes, description, and comment count.  
* The 'category_id' varies bewteen regions with the specific category in associated JSON files.

### Method
In this project, **Spark** will be used for data preparation and data wrangling, **Python** will be used for data visulization.

In [3]:
#Import data and drop useless columns
us = spark.read.csv('/FileStore/tables/USvideos.csv', header="true", inferSchema="true").drop('description','comment_disabled','ratings_disabled','video_error_or_removed')
ca = spark.read.csv('/FileStore/tables/CAvideos.csv', header="true", inferSchema="true").drop('description','comment_disabled','ratings_disabled','video_error_or_removed')
display(us)

video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled
2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False
1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with John Oliver (HBO),LastWeekTonight,24,2017-11-13T07:30:00.000Z,"""last week tonight trump presidency""|""last week tonight donald trump""|""john oliver trump""|""donald trump""",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False
5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Lele Pons",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"""racist superman""|""rudy""|""mancuso""|""king""|""bach""|""racist""|""superman""|""love""|""rudy mancuso poo bear black white official music video""|""iphone x by pineapple""|""lelepons""|""hannahstocking""|""rudymancuso""|""inanna""|""anwar""|""sarkis""|""shots""|""shotsstudios""|""alesso""|""anitta""|""brazil""|""Getting My Driver's License | Lele Pons""",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False
puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"""rhett and link""|""gmm""|""good mythical morning""|""rhett and link good mythical morning""|""good mythical morning rhett and link""|""mythical morning""|""Season 12""|""nickelback lyrics""|""nickelback lyrics real or fake""|""nickelback""|""nickelback songs""|""nickelback song""|""rhett link nickelback""|""gmm nickelback""|""lyrics (website category)""|""nickelback (musical group)""|""rock""|""music""|""lyrics""|""chad kroeger""|""canada""|""music (industry)""|""mythical""|""gmm challenge""|""comedy""|""funny""|""challenge""",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False
d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"""ryan""|""higa""|""higatv""|""nigahiga""|""i dare you""|""idy""|""rhpc""|""dares""|""no truth""|""comments""|""comedy""|""funny""|""stupid""|""fail""",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False
gHZ1Qz0KiKM,17.14.11,2 Weeks with iPhone X,iJustine,28,2017-11-13T19:07:23.000Z,"""ijustine""|""week with iPhone X""|""iphone x""|""apple""|""iphone""|""iphone x review""|""iphone x unboxing""",119180,9763,511,1434,https://i.ytimg.com/vi/gHZ1Qz0KiKM/default.jpg,False
39idVpFF7NQ,17.14.11,Roy Moore & Jeff Sessions Cold Open - SNL,Saturday Night Live,24,2017-11-12T05:37:17.000Z,"""SNL""|""Saturday Night Live""|""SNL Season 43""|""Episode 1730""|""Tiffany Haddish""|""Roy Moore""|""Mikey Day""|""Mike Pence""|""Beck Bennett""|""Jeff Sessions""|""Kate McKinnon""|""s43""|""s43e5""|""episode 5""|""live""|""new york""|""comedy""|""sketch""|""funny""|""hilarious""|""late night""|""host""|""music""|""guest""|""laugh""|""impersonation""|""actor""|""improv""|""musician""|""comedian""|""actress""|""If Loving You Is Wrong""|""Oprah Winfrey""|""OWN""|""Girls Trip""|""The Carmichael Show""|""Keanu""|""Taylor Swift""|""Reputation""|""Look What You Made Me Do""|""ready for it?""|""cold open""",2103417,15993,2445,1970,https://i.ytimg.com/vi/39idVpFF7NQ/default.jpg,False
nc99ccSXST0,17.14.11,5 Ice Cream Gadgets put to the Test,CrazyRussianHacker,28,2017-11-12T21:50:37.000Z,"""5 Ice Cream Gadgets""|""Ice Cream""|""Cream Sandwich Maker""|""gadgets""|""gadget review""|""review""|""unboxing""|""kitchen gadgets""|""Gadgets put to the Test""|""testing""|""10 Kitchen Gadgets""|""7 Camping Coffee Gadgets""|""10 Kitchen Gadgets put to the Test""",817732,23663,778,3432,https://i.ytimg.com/vi/nc99ccSXST0/default.jpg,False
jr9QtXwC9vc,17.14.11,The Greatest Showman | Official Trailer 2 [HD] | 20th Century FOX,20th Century Fox,1,2017-11-13T14:00:23.000Z,"""Trailer""|""Hugh Jackman""|""Michelle Williams""|""Zac Efron""|""Zendaya""|""Rebecca Ferguson""|""pasek and p

In our current dataframe, the category column is in numbers instead of the strings. The according strings are in a nested JSON file. Next, the number category will be converted to the real category name.

In [5]:
#Insert category column
us_cate=spark.read.json("/FileStore/tables/US_category_id.json",multiLine=True)
ca_cate=spark.read.json("/FileStore/tables/CA_category_id.json",multiLine=True)

from pyspark.sql.functions import explode
from pyspark.sql.functions import monotonically_increasing_id

#US area category
us_cate_df=us_cate.select('items.id',"items.snippet.title")
us_id=us_cate_df.select(explode("id").alias("id"))
us_title=us_cate_df.select(explode("title").alias("category"))
us_id = us_id.withColumn("index", monotonically_increasing_id())
us_title = us_title.withColumn("index", monotonically_increasing_id())
df1 = us_title.join(us_id, "index", "outer").drop("index")
us=us.withColumnRenamed('category_id', 'id')
us=us.join(df1,'id',how='left').drop('id')

#CA area category
ca_cate_df=ca_cate.select('items.id',"items.snippet.title")
ca_id=ca_cate_df.select(explode("id").alias("id"))
ca_title=ca_cate_df.select(explode("title").alias("category"))
ca_id = ca_id.withColumn("index", monotonically_increasing_id())
ca_title = ca_title.withColumn("index", monotonically_increasing_id())
df2 = ca_title.join(ca_id, "index", "outer").drop("index")
ca=ca.withColumnRenamed('category_id', 'id')
ca=ca.join(df2,'id',how='left').drop('id')

In [6]:
#combine two datasets into one
df=us.union(ca)
#Check missing values
from pyspark.sql.functions import count
def my_count(df):
  df.agg(*[count(c).alias(c) for c in df.columns]).show()

my_count(df)

+--------+-------------+-----+-------------+------------+-----+-----+-----+--------+-------------+--------------+-----------------+--------+
video_id|trending_date|title|channel_title|publish_time| tags|views|likes|dislikes|comment_count|thumbnail_link|comments_disabled|category|
+--------+-------------+-----+-------------+------------+-----+-----+-----+--------+-------------+--------------+-----------------+--------+
 93697| 83922|83618| 83433| 82816|82096|82007|81981| 81973| 81970| 81970| 81955| 81756|
+--------+-------------+-----+-------------+------------+-----+-----+-----+--------+-------------+--------------+-----------------+--------+

Combine two datasets into one set and adjust the incorrect data type and Check missing values

In [8]:
#drop rows contains missing values
df=df.na.drop()
my_count(df)

+--------+-------------+-----+-------------+------------+-----+-----+-----+--------+-------------+--------------+-----------------+--------+
video_id|trending_date|title|channel_title|publish_time| tags|views|likes|dislikes|comment_count|thumbnail_link|comments_disabled|category|
+--------+-------------+-----+-------------+------------+-----+-----+-----+--------+-------------+--------------+-----------------+--------+
 81756| 81756|81756| 81756| 81756|81756|81756|81756| 81756| 81756| 81756| 81756| 81756|
+--------+-------------+-----+-------------+------------+-----+-----+-----+--------+-------------+--------------+-----------------+--------+

When importing the data, all variables were treated as string. There are some numerical variables, therefore we need to convert it into double datatype.

In [10]:
#transfer data type
from pyspark.sql.functions import col , column, to_date, unix_timestamp
num_col=['views','likes','dislikes','comment_count']
for n in num_col:
  df = df.withColumn(n, col(n).cast("double"))

Add two new variables for future use. One is the ratio of likes over views which can indicates the percentage of likes, one is the ratio of dislikes over views indicating the percentage of dislikes.

In [12]:
import pyspark.sql.functions as func
df=df.withColumn('likes_ratio',func.round(df.likes/df.views,4))
df=df.withColumn('dislikes_ratio',func.round(df.dislikes/df.views,4))
df.printSchema

Out[6]: <bound method DataFrame.printSchema of DataFrame[video_id: string, trending_date: string, title: string, channel_title: string, publish_time: string, tags: string, views: double, likes: double, dislikes: double, comment_count: double, thumbnail_link: string, comments_disabled: string, category: string, likes_ratio: double, dislikes_ratio: double]>

Since the filtered dataset is not quite large, at the later part, it would be transfered into Pandas for the convenience of data visualizaiton.

In [14]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
#transfer into pandas.dataframe
pd_df=df.toPandas() 
pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81756 entries, 0 to 81755
Data columns (total 15 columns):
video_id 81756 non-null object
trending_date 81756 non-null object
title 81756 non-null object
channel_title 81756 non-null object
publish_time 81756 non-null object
tags 81756 non-null object
views 81756 non-null float64
likes 81756 non-null float64
dislikes 81756 non-null float64
comment_count 81756 non-null float64
thumbnail_link 81756 non-null object
comments_disabled 81756 non-null object
category 81756 non-null object
likes_ratio 81756 non-null float64
dislikes_ratio 81756 non-null float64
dtypes: float64(6), object(9)
memory usage: 9.4+ MB

* Considering 'trending_date' and 'publish_time' are datetime variables, it is better to transfer them into datetime type instead of object type. 
* Remove some duplicates records by only keeping the lastest record

In [17]:
#transfer the time datetype
pd_df['trending_date'] = pd.to_datetime(pd_df['trending_date'], format='%y.%d.%m').dt.date
publish_time = pd.to_datetime(pd_df['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')
pd_df['publish_time']=publish_time
#remove duplicates and only keep the lastest record
pd_df=pd_df.sort_values('publish_time',ascending=True).drop_duplicates(subset=['video_id'],keep='last')

In [18]:
#seperate the publish time into date, time and hour for future use
pd_df['publish_date'] = publish_time.dt.date
pd_df['publish_time'] = publish_time.dt.time
pd_df['publish_hour'] = publish_time.dt.hour

### Data Visualazation
#### 1. Correlations between views,likes,dislikes,comment_count,likes_ratio,dislikes_ratio.

In [20]:
corr=pd_df.loc[:,["views","likes","dislikes","comment_count",'likes_ratio','dislikes_ratio']].corr()
ax=sns.heatmap(corr,  vmin=-1, vmax=1, center=0,cmap=sns.diverging_palette(20, 220, n=100),square=True)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
)
plt.title('Heatmap of numeric variables correlations ')
display(ax)

From the correlation plot, we can see all numerical variables are either positive correlated or uncorrelated. 

- 'Dislikes' and 'likes' do not have a significant relation. However 'likes' and 'views' have a strong relation indicating that videos with more views tends to have more likes. But videos with more views does not tend to have more dislikes as well.
- 'likes_ratio' and 'dislikes_ratio' seem uncorrelated with other variables indicating these two variables are quite useful.

#### 2. Which category is always trending?

In [23]:
#category plot
plt.close('all')
fig, ax = plt.subplots()
cat_df_us = pd_df['category'].value_counts().reset_index()
plt.figure(figsize=(15,10))
sns.set_style("whitegrid")
ax=sns.barplot(x='category',y='index',data=cat_df_us)
plt.xlabel("Number of Videos")
plt.ylabel("Categories")
plt.title("Catogories of trend videos")
display(ax)

From the barplot,
* **Top**: Among all the trending videos, the number of Entertainment videos is significantly much higher than the other categories. It is over two times as the second highest category which is News & Politics.  
 + _*This phenomenon aligns with common sense, becuase people watch YouTube for entertainment when they are free in general. News & Politics videos often get trending because there are some breaking news that everyone cares or affect everyone in the area.*_

* **Bottom**: There are few trending videos that belong to Nonprofits & Activism and Movies category. In the plot, one cannot even see it.  
 + _*People do not really watch movies on YouTube and movies are always charged. Besides, movies are too long, no one would click on a three-hour movie even though it is trending. In additions, Nonprofits & Activism is a relatively small genre and thus few trending videos in this category.*_

#### 3. When is the best time in a day to publish video?

In [26]:
plt.close('all')
fig, ax = plt.subplots()
plt.figure(figsize=(15,10))
ax=sns.countplot(x='publish_hour',data=pd_df,palette='Blues_r', order=pd_df['publish_hour'].value_counts().index)
plt.xticks(rotation=90)
plt.xlabel('Publish Hour')
plt.ylabel('Number of Videos')
plt.title('Publish Hour of trend videos')
display(ax)

* In general, most of the trending videos are published in the afternoon or at night
* 15-17 pm seems to be the best time to release a video

#### 4. Top 10 most liked videos
_Found the top 10 videos that have the highest like-ratio_

In [29]:
from IPython.display import HTML
col=['thumbnail_link','title','views','likes_ratio','dislikes_ratio','category']
most_frequent=pd_df[col].sort_values("likes_ratio", ascending=False).head(10)

# Construction of HTML table with miniature photos assigned to the most popular movies
table_content = ''
max_title_length = 50

for date, row in most_frequent.T.iteritems():
    HTML_row = '<tr>'
    HTML_row += '<td><img src="' + str(row[0]) + '"style="width:100px;height:100px;"></td>'
    HTML_row += '<td>' + str(row[1]) + '</td>'
    HTML_row += '<td>' + str(row[2])  + '</td>'
    HTML_row += '<td>' + str(row[3]) + '</td>'
    HTML_row += '<td>' + str(row[4]) + '</td>'
    HTML_row += '<td>' + str(row[5]) + '</td>'
    
    table_content += HTML_row + '</tr>'

displayHTML(
  '<table><tr><th>Thumbnail</th><th>Title</th><th style="width:250px;">Views</th><th>Likes_ratio</th><th>Dislikes_ratio</th><th>Category</th></tr>{}</table>'.format(table_content))

Thumbnail,Title,Views,Likes_ratio,Dislikes_ratio,Category
,The Reaction of The Streets (I Wait-Day6 Edition),88889.0,0.288,0.0001,People & Blogs
,Un Film Pour Raphael,5232.0,0.2632,0.001,Film & Animation
,[MIXTAPE] I.M - Fly With Me (MV),429082.0,0.2492,0.0005,Music
,5 Seconds Of Summer - Want You Back (Audio),439056.0,0.2426,0.0015,Music
,Il fallait que je vous dise...,4739.0,0.2399,0.0006,Comedy
,VOUS ÊTES DÉJÀ ALLÉ EN MUSULMANIE ? (Ramadan voxpop),2242.0,0.2386,0.0027,Comedy
,YOUTUBE DISS TRACK,82487.0,0.2365,0.0025,Gaming
,SLOGOMAN REACTS TO GO GET GONE - KWEBBELKOP & SLOGOMAN DISS TRACK,348156.0,0.2342,0.003,Gaming
,G.C.F in Osaka,2942269.0,0.2341,0.0002,Music
,Day6 Tomfoolery in NY and Japan,98947.0,0.2299,0.0002,People & Blogs


* Trending videos do not always have the most views. Many from the top 10 which have high like-ratio over views got into trending have only a few thousands views. 
* Likes_ratio seems to be an important part of YouTube trending videos Algorithm.
* Most liked videos do not domain in one or two categories, their categories vary.

#### 5. Top 10 most viewd videos
_Found the top 10 videos that have the highest views_

In [32]:
col=['thumbnail_link','title','views','likes_ratio','dislikes_ratio','category']
pd_df[pd_df.category!='Music']
most_frequent=pd_df[col].sort_values("views", ascending=False).head(10)

# Construction of HTML table with miniature photos assigned to the most popular movies
table_content = ''
max_title_length = 50

for date, row in most_frequent.T.iteritems():
    HTML_row = '<tr>'
    HTML_row += '<td><img src="' + str(row[0]) + '"style="width:100px;height:100px;"></td>'
    HTML_row += '<td>' + str(row[1]) + '</td>'
    HTML_row += '<td>' + str(row[2])  + '</td>'
    HTML_row += '<td>' + str(row[3]) + '</td>'
    HTML_row += '<td>' + str(row[4]) + '</td>'
    HTML_row += '<td>' + str(row[5]) + '</td>'
    
    table_content += HTML_row + '</tr>'

displayHTML(
  '<table><tr><th>Thumbnail</th><th>Title</th><th style="width:250px;">Views</th><th>Likes_ratio</th><th>Dislikes_ratio</th><th>Category</th></tr>{}</table>'.format(table_content))

Thumbnail,Title,Views,Likes_ratio,Dislikes_ratio,Category
,Childish Gambino - This Is America (Official Video),210338856.0,0.023,0.0016,Music
,Ariana Grande - No Tears Left To Cry,101386547.0,0.0275,0.0011,Music
,Marvel Studios' Avengers: Infinity War Official Trailer,84281319.0,0.0303,0.0006,Entertainment
,"Luis Fonsi, Demi Lovato - Échame La Culpa",68997838.0,0.0297,0.0014,Music
,"Calvin Harris, Dua Lipa - One Kiss (Official Video)",62800938.0,0.0124,0.0005,Music
,BTS (방탄소년단) 'FAKE LOVE' Official MV,62796390.0,0.0712,0.0019,Music
,Nicky Jam x J. Balvin - X (EQUIS) | Video Oficial | Prod. Afro Bros & Jeon,61163906.0,0.0103,0.0004,Music
,Drake - Nice For What,57582925.0,0.0169,0.0006,Music
,To Our Daughter,56111957.0,0.0,0.0,People & Blogs
,Ed Sheeran - Happier (Official Video),51747671.0,0.0263,0.0006,Music


* Most of the most viewed trending videos are music vedios
* When the videos get millions of views, the like ratio is not referable anymore
* Videos from popular singers or popular movie trailer can attract most of the users to click on and tend to get trending

In [34]:
%sh python -m nltk.downloader all

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
 warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'all'
[nltk_data] | 
[nltk_data] | Downloading package abc to /root/nltk_data...
[nltk_data] | Unzipping corpora/abc.zip.
[nltk_data] | Downloading package alpino to /root/nltk_data...
[nltk_data] | Unzipping corpora/alpino.zip.
[nltk_data] | Downloading package biocreative_ppi to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/biocreative_ppi.zip.
[nltk_data] | Downloading package brown to /root/nltk_data...
[nltk_data] | Unzipping corpora/brown.zip.
[nltk_data] | Downloading package brown_tei to /root/nltk_data...
[nltk_data] | Unzipping corpora/brown_tei.zip.
[nltk_data] | Downloading package cess_cat to /root/nltk_data...
[nltk_data] | Unzipping corpora/cess_cat.zip.
[nltk_data] | Downloading package cess_esp to /root/nltk_data...
[nltk_data] | Unzipping corpora/cess_esp.zip.
[nltk_data] | Downloading package chat80 to /root/nltk_data...
[nltk_data] | Unzipping corpora/chat80.zip.
[nltk_data] | Downloading package city_database to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/city_database.zip.
[nltk_data] | Downloading package cmudict to /root/nltk_data...
[nltk_data] | Unzipping corpora/cmudict.zip.
[nltk_data] | Downloading package comparative_sentences to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/comparative_sentences.zip.
[nltk_data] | Downloading package comtrans to /root/nltk_data...
[nltk_data] | Downloading package conll2000 to /root/nltk_data...
[nltk_data] | Unzipping corpora/conll2000.zip.
[nltk_data] | Downloading package conll2002 to /root/nltk_data...
[nltk_data] | Unzipping corpora/conll2002.zip.
[nltk_data] | Downloading package conll2007 to /root/nltk_data...
[nltk_data] | Downloading package crubadan to /root/nltk_data...
[nltk_data] | Unzipping corpora/crubadan.zip.
[nltk_data] | Downloading package dependency_treebank to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/dependency_treebank.zip.
[nltk_data] | Downloading package dolch to /root/nltk_data...
[nltk_data] | Unzipping corpora/dolch.zip.
[nltk_data] | Downloading package europarl_raw to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/europarl_raw.zip.
[nltk_data] | Downloading package floresta to /root/nltk_data...
[nltk_data] | Unzipping corpora/floresta.zip.
[nltk_data] | Downloading package framenet_v15 to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/framenet_v15.zip.
[nltk_data] | Downloading package framenet_v17 to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/framenet_v17.zip.
[nltk_data] | Downloading package gazetteers to /root/nltk_data...
[nltk_data] | Unzipping corpora/gazetteers.zip.
[nltk_data] | Downloading package genesis to /root/nltk_data...
[nltk_data] | Unzipping corpora/genesis.zip.
[nltk_data] | Downloading package gutenberg to /root/nltk_data...
[nltk_data] | Unzipping corpora/gutenberg.zip.
[nltk_data] | Downloading package ieer to /root/nltk_data...
[nltk_data] | Unzipping corpora/ieer.zip.
[nltk_data] | Downloading package inaugural to /root/nltk_data...
[nltk_data] | Unzipping corpora/inaugural.zip.
[nltk_data] | Downloading package indian to /root/nltk_data...
[nltk_data] | Unzipping corpora/indian.zip.
[nltk_data] | Downloading package jeita to /root/nltk_data...
[nltk_data] | Downloading package kimmo to /root/nltk_data...
[nltk_data] | Unzipping corpora/kimmo.zip.
[nltk_data] | Downloading package knbc to /root/nltk_data...
[nltk_data] | Downloading package lin_thesaurus to
[nltk_data] | /root/nltk_data...
[nltk_data] | Unzipping corpora/lin_thesaurus.zip.
[nltk_data] | Downloading package mac_morpho to /root/nltk_data...
[nltk_data] | Unzipping corpora/mac_morpho.zip.
[nltk_data] | Downloading package machado to /root/n

#### 6. What insight does TAGS show

In [36]:
#wordcloud
plt.close('all')
from wordcloud import WordCloud 
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


#tags_word = pd_df[pd_df['category']=='News & Politics']['tags'].str.lower().str.cat(sep=' ')
tags_word = pd_df['tags'].str.lower().str.cat(sep=' ')
tags_word = re.sub('[^A-Za-z]+', ' ', tags_word)
word_tokens = word_tokenize(tags_word)
en_stopwords= set(stopwords.words('english'))
filtered_sentence = [w for w in word_tokens if not w in en_stopwords]
without_single_chr = [word for word in filtered_sentence if len(word) > 2]
cleaned_data_title = [word for word in without_single_chr if not word.isdigit()]

from collections import Counter
counts=Counter(cleaned_data_title)

wordcloud = WordCloud(width=900,height=500, max_words=3000,relative_scaling=1).generate_from_frequencies(counts)

plt.figure(figsize=(14, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
display()

The size of the word in the wordcloud is based on the frequency of a word in the tags. The higher frequency, the bigger the word size is.  
<br>

 * 'News' and 'Trump' apprently happen very frequently in the news&politics category videos and news&politics do have the second highest number among all the trending videos.
 * Same for the music category with the 'songs' and 'music'
 * Positive words such as 'funny', 'comedy', 'family', 'best', 'challenge', 'highlights' are quite frequent among trending videos.

### Recomendations to YouTubers  
<br>
- **Do ask your viewers to give likes to the video** becuase it can still push your video to be trending regardless of how many views your video have
- **Publish your video during 15-17 pm of your day** because most of the trending videos are released during that time
- **Tag your video with positive words** because the most frequent words are either neutural function words or positive words